# FTX reader

In [15]:
import matplotlib
%matplotlib inline
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px

import sys
sys.path.append('../')
from history_script import *

import os
directory='C:/Users/david/puffin/DerivativeArbitrage/Runtime/live_parquets/'
end_time=datetime.today().replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(days=7)
modes=[('h','high'),('c','close'),('l','low')]
coins=['OKB','1INCH']

In [16]:
futures_df=ftx_read_history(dirname=directory,coin_list=coins)

{'apiKey': True, 'secret': True, 'uid': False, 'login': False, 'password': False, 'twofa': False, 'privateKey': False, 'walletAddress': False, 'token': False}


,1INCH-PERP/rate/funding,1INCH-PERP/rate/funding,OKB-PERP/rate/funding,OKB-PERP/rate/funding,1INCH-PERP/mark/o,1INCH-PERP/mark/h,1INCH-PERP/mark/l,1INCH-PERP/mark/c,1INCH-PERP/mark/volume,1INCH-PERP/indexes/open,...,USD/rate/size,USD/rate/borrow,1INCH/rate/borrow,1INCH/rate/size,1INCH/rate/borrow,1INCH/rate/size,OKB/rate/borrow,OKB/rate/size,OKB/rate/borrow,OKB/rate/size


In [17]:
prices=futures_df[[c for c in futures_df.columns if any(string in c for string in ['mark/c','price/c'])]]
rates=futures_df[[c for c in futures_df.columns if any(string in c for string in ['rate/c','rate/borrow'])]]
prices.join(rates).iplot(secondary_y=list(rates.columns))

## stopout: show min/max of premiums

In [4]:
def compute_premium(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/indexes/'+modes[mode][1]).iloc[:,0]-1)*10000
def compile_list(premium_list: list) -> pd.DataFrame:
    premium=premium_list[0]
    for df in premium_list[1:]:
        premium=premium.join(df,how='outer')
    return premium[~premium.index.duplicated()]

In [5]:
futures_df['premium']=compute_premium(futures_df)

In [6]:
premium=compute_premium(futures_df,mode=0)
ewMax=premium.resample('1h').max()
ewMax=ewMax[~ewMax.index.duplicated()]
fig = px.line(ewMax,title="fut high - spot high")#, x='date', y="GOOG")
fig.show()

In [7]:
premium=compute_premium(futures_df,mode=2)
ewMin=premium.resample('1h').min()
ewMin=ewMin[~ewMin.index.duplicated()]
fig = px.line(ewMin,title="fut low - spot low")#, x='date', y="GOOG")
fig.show()

## spot vs funding